In [12]:


class Concept:
    def __init__(self, concept_name, metadata):
        self.name = concept_name
        self.metadata = metadata

class Topic:
    def __init__(self, name):
        self.name = name
        self.concepts = []

    def add_concept(self, concept):
        self.concepts.append(concept)

class MetaCovidOntology:
    def __init__(self, owl_file_path):
        from rdflib.namespace import Namespace
        
        self.owl_file_path = owl_file_path
        
        # Define namespaces within the class
        self.BASE = Namespace("http://example.org/metacovid/")
        self.UMLS = Namespace("http://example.org/metacovid/UMLS/")
        self.CAP = Namespace("http://example.org/metacovid/CAP/")
        self.EUROVOC = Namespace("http://example.org/metacovid/Eurovoc/")
        self.METADATA = Namespace("http://example.org/metacovid/metadata/")
        
        self.metadata = {}
        self.topics = []
        self._load_data()

    def _load_data(self):
        from rdflib import Graph, RDF, RDFS, OWL
        
        g = Graph()
        g.parse(self.owl_file_path)

        # Extract metadata
        for ontology in ["UMLS", "CAP", "EuroVoc"]:
            ontology_namespace = None
            if ontology == "UMLS":
                ontology_namespace = self.UMLS["Metadata"]
            elif ontology == "CAP":
                ontology_namespace = self.CAP["Metadata"]
            elif ontology == "EuroVoc":
                ontology_namespace = self.EUROVOC["Metadata"]
            
            self.metadata[ontology] = {}
            for pred, obj in g.predicate_objects(subject=ontology_namespace):
                pred_name = str(pred).split("/")[-1]
                if pred_name != "22-rdf-syntax-ns#type":
                    self.metadata[ontology][pred_name] = str(obj)

        # Extract topics and associated concepts
        for subject in g.subjects(RDF.type, OWL.Class):
            topic_name = str(subject).split("/")[-1].replace("_", " ")
            topic_obj = Topic(topic_name)
            
            for individual in g.subjects(RDF.type, subject):
                concept_name = None
                concept_metadata = {}
                for pred, obj in g.predicate_objects(subject=individual):
                    pred_name = str(pred).split("/")[-1]
                    # Exclude the '22-rdf-syntax-ns#type' key for concepts as well
                    if pred_name != "22-rdf-syntax-ns#type":
                        if pred == RDFS.label:
                            concept_name = str(obj)
                        else:
                            concept_metadata[pred_name] = str(obj)
                                    # Set the ontology to which the concept belongs in its metadata
                if "UMLS" in str(individual):
                    concept_metadata["ontology"] = "UMLS"
                elif "CAP" in str(individual):
                    concept_metadata["ontology"] = "CAP"
                elif "EuroVoc" in str(individual):
                    concept_metadata["ontology"] = "EuroVoc"
                
                topic_obj.add_concept(Concept(concept_name, concept_metadata))
                
            # Append the topic to the topics list
            self.topics.append(topic_obj)

        # sort topics by MC index
        resort_indices = sorted(range(len(self.topics)), key=lambda x: int(self.topics[x].name.split(':')[0][2:])) 
        self.topics = [self.topics[i] for i in resort_indices]

    def get_metadata(self):
        return self.metadata

    def get_topics(self):
        return self.topics

In [13]:
# Create an instance of MetaCovidOntology class
owl_file_path = 'metacovid_ontology.owl'
metacovid_ontology = MetaCovidOntology(owl_file_path)

# For demonstration purposes, we'll fetch and display the metadata and the name of the first topic
demo_metadata = metacovid_ontology.get_metadata()
demo_first_topic_name = metacovid_ontology.get_topics()[0].name

In [14]:
topics = metacovid_ontology.topics
topics
topics[0].concepts[3].name

'food price'

In [15]:
sum(len(topic.concepts) for topic in topics)

203

In [16]:
demo_metadata

{'UMLS': {'Resource_name': 'Unified Medical Language System (UMLS) Metathesaurus',
  'Version': '2023AA',
  'URL': 'https://www.nlm.nih.gov/research/umls',
  'Description': 'The UMLS Metathesaurus is a large biomedical thesaurus that is organized by concept, or meaning. It links synonymous names from over 200 different source vocabularies. The Metathesaurus also identifies useful relationships between concepts and preserves the meanings, concept names, and relationships from each vocabulary.'},
 'CAP': {'Resource_name': 'Comparative Agendas Project (CAP): Master Topics Codebook',
  'Release': 'Version 1.1',
  'URL': 'https://www.comparativeagendas.net/pages/master-codebook',
  'Description': 'Observations in Comparative Agendas Project datasets are coded according to the guidelines and topic system described below. This codebook is an updated version of the original codebook created by Baumgartner and Jones.  Each entry is coded into one of 21 major topics and 220 subtopics.  In this c

In [17]:
[concept.name for concept in metacovid_ontology.topics]

['MC1: Agriculture, Food, & Sustainability',
 'MC2: Business Impact & Sector-specific Challenges',
 'MC3: Clinical Treatment & Hospital Care',
 'MC4: Communication & Information Dissemination',
 'MC5: Consumer Behavior & Lifestyle Changes',
 'MC6: Defense & National Security',
 'MC7: Digital Transformation & Technology',
 'MC8: Economic Impact & Recovery',
 'MC9: Environment & Climate',
 'MC10: Epidemiology & Virus Transmission',
 'MC11: Ethics of Healthcare & Treatment',
 'MC12: Gender & Minority Groups',
 'MC13: Government Policy & Response',
 'MC14: International Collaboration & Global Initiatives',
 'MC15: Labor & Employment Dynamics',
 'MC16: Legal & Human Rights',
 'MC17: Media, Public Perception & Culture',
 'MC18: Mental Health & Psychological Well-being',
 'MC19: Migration & Movement Patterns',
 'MC20: Political Implications & Governance',
 'MC21: Public Health & Preventative Measures',
 'MC22: Research, Innovation & Development',
 'MC23: Safety & Security Protocols',
 'MC24: 

In [20]:
topics[0].name

'MC1: Agriculture, Food, & Sustainability'